# Feature Engineering Tasks

From the tasks, it's apparent the data is being prepared for some ML analysis, therefore the tasks are performed keeping this objective in mind. However, the dataset is not being partitioned into the testing and training sets as it is not a requirement in the task.

## Importing Relevant Libraries and Dataset

In [121]:
# Importing Libraries

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler

In [122]:
# Load the dataset
data = pd.read_csv("../data/titanic.csv")

# Make a copy of the dataset
copy = data.copy()